In [2]:
import requests
import json
import csv

In [3]:
class Patients:
    def __init__(self, pid, type, value,reference):
        self.pid = pid
        self.type = type
        self.value = value
        self.ref = reference

    def __str__(self):
        return "pid:{}, type:{},value: {}".format(self.pid,self.type,self.value)

In [4]:
OAuthurl = 'https://sandbox86.tactiorpm7000.com/token.php'
APIurl = 'https://sandbox86.tactiorpm7000.com/tactio-clinician-api/1.1.4'

# '''Authentication'''
body = {
"grant_type": "password",
"client_id": "083e9a44a763473fbeb62fbf90b74551",
"client_secret": "ba09798f0921456e8b4e5e4588ea536d",
"username": "tactioClinician",
"password": "tactio"
}

response = requests.post(OAuthurl, data=body)
auth_code = response.json()["access_token"]

In [5]:
# initialize values (patients_list == type.dict)
patient_dict = {}

### add BMI to patient_dict

In [6]:
observation_param = [("code", "39156-5"),("code", "39156-5")]
patient_observation_raw = requests.get(APIurl + '/Observation', headers={
    'Authorization': 'Bearer {}'.format(auth_code)}, params=observation_param)
patient_obs_json = patient_observation_raw.json()

for patient in patient_obs_json['entry']:
    # get the patient name id
    patient_name_id = patient['resource']['subject']['reference']
    # if patient_dict does not have the name, create a empty key
    if (patient_name_id not in patient_dict):
        # request every patient's info (name, gender...)
        patient_info_raw = requests.get(APIurl+"/{}".format(patient_name_id), headers={
        'Authorization': 'Bearer {}'.format(auth_code)})
        
        name = str(patient_info_raw.json()['name'][0]['given'][0])+str(patient_info_raw.json()['name'][0]['family'])
        patient_dict[patient_name_id] = {}
        patient_dict[patient_name_id]['name'] = {name}
        
    # add bmi to key
    patient_dict[patient_name_id]['bmi'] = patient['resource']['valueQuantity']['value']

In [7]:
observation_param = [("code", "73708-0"),("code", "73708-0")]
patient_observation_raw = requests.get(APIurl + '/Observation', headers={
    'Authorization': 'Bearer {}'.format(auth_code)}, params=observation_param)
patient_obs_json = patient_observation_raw.json()

for patient in patient_obs_json['entry']:
    # get the patient name id
    patient_name_id = patient['resource']['subject']['reference']
    # if patient_dict does not have the name, create a empty key
    if (patient_name_id not in patient_dict):
        # request every patient's info (name, gender...)
        patient_info_raw = requests.get(APIurl+"/{}".format(patient_name_id), headers={
        'Authorization': 'Bearer {}'.format(auth_code)})
        
        name = str(patient_info_raw.json()['name'][0]['given'][0])+str(patient_info_raw.json()['name'][0]['family'])
        patient_dict[patient_name_id] = {}
        patient_dict[patient_name_id]['name'] = {name}
        
    # add bmi to key
    patient_dict[patient_name_id]['fat_ratio'] = patient['resource']['valueQuantity']['value']

In [16]:
# Write to csv
with open("dict.txt", 'w',  newline='') as outfile:
    csv_writer = csv.writer(outfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for k,v in patient_dict.items():
        
        # attributes
        name = ' '
        bmi = ' '
        fat_ratio = ' '
        
        if ('name' in v):
            name = v['name']
        if ('bmi' in v):
            bmi = v['bmi']
        if ('fat_ratio' in v):
            fat_ratio = v['fat_ratio']
        
        #csv_writer.writerow([k] + [name]  + [bmi] + [fat_ratio])
        csv_writer.writerow([k] + [bmi] + [fat_ratio])

In [ ]:
patient_dict